<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex15notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex15notecookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# 「1000種類の物体を識別するニューラルネットを動かしてみよう」のためのほげ
import pickle
import torch
from torchvision import models, transforms
from PIL import Image
import json

---
## 手書き数字識別の例
---

授業で学んだことを実際のデータで理解するための例です．

---
### 準備



これまでに何回も登場したものと同様の，28x28画素の手書き数字のデータです．

In [ ]:
## 手書き数字データの入手

# 一つ目のデータセット
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
datL = rv['datL'].astype(float)[:2500, :]
labL = rv['labL'][:2500]
datT = rv['datT'].astype(float)[:2500, :]
labT = rv['labT'][:2500]
print(datL.shape, labL.shape, datT.shape, labT.shape)

# 二つ目のデータセット
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist_biased.npz
rv = np.load('minimnist_biased.npz')
datLb = rv['datL'].astype(float)
labLb = rv['labL']
datTb = rv['datT'].astype(float)
labTb = rv['labT']
print(datLb.shape, labLb.shape, datTb.shape, labTb.shape)

K = 10 # クラス数
D = datL.shape[1] # データの次元数 28 x 28 = 784

In [ ]:
# データを画像として表示するための関数
#
def display(data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix < data.shape[0]:
                ltx = ix*(ncol + gap) + gap
                img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    # 画像の出力
    plt.axis('off')
    plt.imshow(img, cmap = 'gray')
    plt.show()

---
### 一つ目のデータセットで識別実験

一つ目のデータセットに最近傍法を適用し，テストデータを識別させてみましょう．

学習データはこんなんです．

In [ ]:
nx, ny = 10, 5
display(datL[:50], nx, ny)

In [ ]:
for iy in range(ny):
    print(labL[iy*nx:(iy+1)*nx])

クラスごとに正解率を出してみると...




In [ ]:
# 予測されたクラス番号を格納する配列
predT = np.empty_like(labT)
NT, D = datT.shape

# 最近傍法で識別
for n in range(NT):
    d2 = np.sum((datT[n, :] - datL)**2, axis=1)
    predT[n] = labL[np.argmin(d2)]

# クラスごとの正解率を算出
nca = 0
for ic in range(K):
    idx = labT == ic
    nc = np.sum(predT[idx] == ic)
    nn = np.sum(idx)
    print(f'class{ic}: {nc}/{nn}')
    nca += nc

print(f'total: {nca}/{NT}')

こんなん出ました．

---
### 二つ目のデータセットで識別実験

二つ目のデータセットに最近傍法を適用し，テストデータを識別させてみましょう．
二つのデータセットは，学習データのみ異なり，テストデータは全く同じものです．

学習データはこんなんです．
一つ目と同じようなものに見えます．データ数も同じです．

In [ ]:
nx, ny = 10, 5
display(datLb[:50], nx, ny)

In [ ]:
for iy in range(ny):
    print(labLb[iy*nx:(iy+1)*nx])

しかし，識別させてみると，次のような結果になります．

In [ ]:
# 予測されたクラス番号を格納する配列
predTb = np.empty_like(labT)
NT, D = datT.shape

# 最近傍法で識別
for n in range(NT):
    d2 = np.sum((datT[n, :] - datLb)**2, axis=1)
    predTb[n] = labLb[np.argmin(d2)]

# クラスごとの正解率を算出
nca = 0
for ic in range(K):
    idx = labT == ic
    nc = np.sum(predTb[idx] == ic)
    nn = np.sum(idx)
    print(f'class{ic}: {nc}/{nn}')
    nca += nc

print(f'total: {nca}/{NT}')

全体の正解率が下がっていますが，特に class2 と class9 の精度が低いようです．
正解率の低いクラスでどんな間違い方をしているか，詳しく見てみましょう．

In [ ]:
# 正解が class2 であるテストデータのうち，予測を間違えたもの
idx = (labT == 2) & (predTb != 2)
XX = datTb[idx, :]
YY = predTb[idx]
nx, ny = 10, 4
display(XX[:nx*ny], nx, ny)
for iy in range(ny):
    print(YY[iy*nx:(iy+1)*nx])

上記は，正解が class2 であるテストデータのうち，予測を間違えたものをすべて並べてあります．
そりゃ間違えても仕方ないな，という例もありますが，普通に読めるきれいな「2」もたくさん間違えているようです．

In [ ]:
# 正解が class9 であるテストデータのうち，予測を間違えたもの
idx = (labT == 9) & (predTb != 9)
XX = datTb[idx, :]
YY = predTb[idx]
nx, ny = 10, 4
display(XX[:nx*ny], nx, ny)
for iy in range(ny):
    print(YY[iy*nx:(iy+1)*nx])

同様に，正解が class9 であるテストデータのうち，予測を間違えたものは上記の通りです．
やはり，まともそうな「9」をたくさん間違えているようです．

---
### 学習データを眺めて考えてみよう

どうして上記のような実験結果となるのか，学習データを眺めて理由を考えましょう．

In [ ]:
# 一つ目のデータセット
idx = labL == 2
XX = datL[idx, :]
nx, ny = 16, 10
display(XX[:nx*ny], nx, ny)

In [ ]:
# 二つ目のデータセット
idx = labLb == 2
XX = datLb[idx, :]
nx, ny = 16, 10
display(XX[:nx*ny], nx, ny)

In [ ]:
# 一つ目のデータセット
idx = labL == 9
XX = datL[idx, :]
nx, ny = 16, 10
display(XX[:nx*ny], nx, ny)

In [ ]:
# 二つ目のデータセット
idx = labLb == 9
XX = datLb[idx, :]
nx, ny = 16, 10
display(XX[:nx*ny], nx, ny)

二つ目のデータセットでの識別実験で間違えてるのはどんな画像だったかを見返しながら上の二つの画像を比較するとよいでしょう．答えを知りたければ，takataka に尋ねてください．ヒント: これらは，「機械学習と公平性」の話と関係しています．

---
## 1000種類の物体を識別するニューラルネットを動かしてみよう
---

ニューラルネットを用いた画像識別の実験をやってみましょう． いろんなものが識別できた方が楽しいと思いますので，1000種類の物体を識別させてみます． ただし，そういうことができるニューラルネットワークを一から学習させようとすると大変ですので，ここではすでに学習済みのものを利用することにします．

---
### 準備

まずは実験に使う画像データを準備します．

**注意: 以下の画像をこの実験以外の目的で使用してはいけません**

In [ ]:
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/adversarialphoto.pickle
with open('adversarialphoto.pickle', 'rb') as f:
    hoge = pickle.load(f)

imgList = []
for i, d in enumerate(hoge):
    img = Image.frombytes(**d)
    imgList.append(img)
    if i >= 4:
        continue
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    ax.axis('off')
    plt.show()

1000個あるカテゴリの番号とそのラベルの対応表を作成します．

In [ ]:
# ImageNet カテゴリラベルを表すJSONファイルを入手
!wget -nc https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
class_index = json.load(open('imagenet_class_index.json', 'r'))

K = len(class_index)
labels = {}
for ik, key in enumerate(class_index.keys()):
    labels[ik] = f'{class_index[key][0]} {class_index[key][1]}'
    if 276 <= ik < 300:
        print(ik, labels[ik])

上記の出力は，1000カテゴリのうちの一部のものの名前を表します．「ネコ」みたいなものも1カテゴリではなく，281番 'tabby'（ぶち柄の猫），282番 'tiger_cat'（トラ縞の猫）等々のように分かれています．

学習済みのニューラルネットワークのパラメータを入手します．
規模の大きいネットワークでパラメータがたくさんありますので，読み込みに少し時間がかかります．

In [ ]:
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
vgg16.eval()
print(vgg16)

上記のセルを実行すると，次のことが行われます．
1. ニューラルネットの学習モデルを作成する．
1. 学習済みのネットワークパラメータの値を読み込んでこのネットワークのパラメータに設定する．
1. ネットワークモデルの構造を表示する．

参考:

ここで用いているのは VGG16 というニューラルネットモデルです．イギリスオックスフォード大学の [Visual Geometry Group(VGG)](https://www.robots.ox.ac.uk/~vgg/) という研究グループが作った VGG-net というニューラルネットモデルのうちの，層が16層あるものです．
VGG-net は，2014年に行われた [ILSVRC2014](https://www.image-net.org/challenges/LSVRC/2014/) という画像識別のコンペティションで世界第2位となったニューラルネットモデルです．
ILSVRC2014では，[ImageNet](https://www.image-net.org/) という大規模な画像データセットの中から選ばれた 1000 種類の物体の画像約120万枚を学習データとしていました．


画像をネットワークに入力できる形式に加工する処理を定義しておきます．


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

---
### 実験その1

上記のサンプル画像たちを VGG16 に識別させてみましょう．


In [ ]:
#@markdown 以下で 0 から 3 までの整数を選ぶと4つのサンプル画像の中から一つを選ぶことができます．
i = 0 #@param [0, 1, 2, 3] {type: 'raw'}
img = imgList[i]

# 画像を表示
fig, ax = plt.subplots(1)
ax.imshow(img)
ax.axis('off')
plt.show()

# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(imgList[i]), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')

上記の画像の下の出力は，1000個のカテゴリの中で，ネットワークの出力の値が大きかったもの上位5位までの，出力の値とカテゴリラベルを表します．
出力の値は，そのカテゴリと判定することの「確信度」と解釈できます．

---
### 実験その2

上記のサンプル画像は4枚ありましたが，実は他にまだ4枚あります．
次のセルで `i` を 4 から 7 まで選ぶと画像とニューラルネットの出力を確認できます．

In [ ]:
#@markdown 以下で 0 から 7 までの整数を選ぶと8つのサンプル画像の中から一つを選ぶことができます．
i = 4 #@param [0, 1, 2, 3, 4, 5, 6, 7] {type: 'raw'}
img = imgList[i]

# 画像を表示
fig, ax = plt.subplots(1)
ax.imshow(img)
ax.axis('off')
plt.show()

# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(imgList[i]), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')

`i` を 4 から 7 にしたときの画像は，「再現性・ロバスト性」の話の中に登場した「敵対的事例」のサンプルです．VGG16モデルが，0番から999番まであるクラスのうち999番の

In [ ]:
labels[999]

と誤識別しやすくなる方向に画像を少し歪ませたものです．その結果どんなクラスと間違えているか，出力を確認してみてください．

［補足］
- ここで用いている画像例は，元画像がかなり高い確信度で正しいクラスに識別されている例ばかりです．そのため，誤識別させるためにかなり大きく歪ませており，目で見て分かるほどになってしまっています．
実際には，人間には気づかないほどの小さな歪を画像に混入させるだけで識別結果を捻じ曲げられる場合もあります．
- ここでは，Fast Gradient Sign Method と呼ばれる手法で敵対的事例を作っています．参考: https://pytorch.org/tutorials/beginner/fgsm_tutorial.html

---
### 実験その3

自分で用意した画像でも実験してみましょう．

- 上記では動物の例ばかりですが，1000のカテゴリの中にはそれ以外にも様々なものがあります（「人間」というカテゴリはありません．どうなる？）
- 面白い／不思議な結果が得られたら takataka に見せてくれると喜びます



(1) まずは，ウェブ等で適当な画像を探して，自分のコンピュータへ保存しましょう．ファイル名が長かったり日本語を含んでいる場合は，短い名前に変更しておくのがよいです．

(2) 以下のセルを実行して，ファイルをアップロードします．

In [ ]:
# Colab へファイルをアップロード
try:
    from google.colab import files
    rv = files.upload()
except:
    print('このコードは Colab 以外の環境では実行できないよ．')

(3) 以下のセルの1行目のファイル名を上記でアップロードしたものに変えて実行しましょう．アルファベット大文字小文字の区別もありますので注意．うまくいけば画像が表示されるはずです．

In [ ]:
# ↓のファイル名をアップロードしたものに変更してね
myimg = Image.open('hoge.png')
if myimg.mode != 'RGB':
    myimg = myimg.convert('RGB')
fig, ax = plt.subplots(1)
ax.imshow(myimg)
ax.axis('off')
plt.show()

(4) 以下のセルを実行すれば結果が表示されます．

In [ ]:
# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(myimg), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')